In [1]:
from torchvision import utils
from basic_fcn import *
from dataloader import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time
import matplotlib.pyplot as plt

In [2]:
batch = 2
train_dataset = CityScapesDataset(csv_file='train.csv')
val_dataset = CityScapesDataset(csv_file='val.csv')
test_dataset = CityScapesDataset(csv_file='test.csv')
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch,
                          num_workers=4,
                          shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=batch,
                          num_workers=4,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=batch,
                          num_workers=4,
                          shuffle=True)

In [3]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        torch.nn.init.xavier_uniform_(m.bias.data.unsqueeze(0)) #add unsqueeze here 

In [4]:
def weighted_cross_entropy(x, y, weights):
    probability = -1.0 * F.log_softmax(x, dim = 1)
    
    loss = probability.gather(1, y.unsqueeze(1))
    return loss.mean()

In [5]:
x = torch.rand(34).cuda()
print(x)
criterion = nn.CrossEntropyLoss(weight=x)
criterion_unweighted = nn.CrossEntropyLoss()
x = x.view(1, 34, 1, 1).cuda()
fcn_test = FCN(n_class=n_class)
fcn_test.apply(init_weights)
fcn_test = fcn_test.cuda()

tensor([0.0994, 0.1993, 0.0520, 0.1805, 0.3327, 0.3999, 0.4218, 0.3714, 0.3637,
        0.6260, 0.6885, 0.9683, 0.5666, 0.5976, 0.7129, 0.5414, 0.6813, 0.7918,
        0.2177, 0.4594, 0.8826, 0.9867, 0.5193, 0.4215, 0.6516, 0.0564, 0.5730,
        0.6652, 0.2092, 0.8599, 0.9266, 0.8547, 0.3839, 0.2919],
       device='cuda:0')


In [6]:
# Test weighted cross-entropy
fcn_test.eval()
for i_batch, sample_batched in enumerate(train_loader):
    inputs = sample_batched[0].cuda()
    one_hot_labels = sample_batched[1].cuda()
    labels = sample_batched[2].cuda()
    outputs = fcn_test(inputs)
    del inputs
    
    probability = -1.0 * F.log_softmax(outputs, dim = 1)
    weighted_probability = probability*x
    loss_weighted_self = weighted_probability.gather(1, labels.unsqueeze(1))
    loss_weighted_self = loss_weighted_self.mean()
    print("My weighted loss function outputs:", loss_weighted_self.item())
    
    loss_unweighted_self = probability.gather(1, labels.unsqueeze(1))
    loss_unweighted_self = loss_unweighted_self.mean()
    print("My unweighted loss function outputs:", loss_unweighted_self.item())
    
    loss_weighted = criterion(outputs, labels)
    print("PyTorches weighted loss function outputs:", loss_weighted.item())
    
    loss_unweighted = criterion_unweighted(outputs, labels)
    print("PyTorches unweighted loss function outputs:", loss_unweighted.item())
      
    del one_hot_labels, labels, outputs, probability, weighted_probability, loss_unweighted, loss_unweighted_self, loss_weighted, loss_weighted_self
    break

My weighted loss function outputs: 2.1941280364990234
My unweighted loss function outputs: 3.605480670928955
PyTorches weighted loss function outputs: 3.5787203311920166
PyTorches unweighted loss function outputs: 3.605480432510376


In [7]:
print("Current memory allocation:", torch.cuda.memory_allocated())

Current memory allocation: 22948864
